In [105]:
from diel_models.pipeline import Pipeline
from diel_models.pipeline_intermediate import DayNightCreator, BiomassRegulator, NitrateUptakeRatioCalibrator, StoragePoolGenerator, PhotonReactionInhibitor
import os
import cobra
from tests import TEST_DIR

aragem_model_path = os.path.join(TEST_DIR, 'data', 'AraGEM2010.xml')

modelo = cobra.io.read_sbml_model(aragem_model_path)

Steps = [
    DayNightCreator(modelo),
    StoragePoolGenerator(modelo, metabolites = ["S_Sucrose_c_Day", "S_Sulfate_c_Day", "S_Nitrate_c_Day", "S_L_45_Histidine_c_Day", "S_L_45_Isoleucine_c_Day", "S_L_45_Leucine_c_Day", "S_L_45_Lysine_c_Day", "S_L_45_Methionine_c_Day", "S_L_45_Phenylalanine_c_Day", "S_L_45_Threonine_c_Day", "S_L_45_Tryptophan_c_Day", "S_L_45_Valine_c_Day", "S_L_45_Arginine_c_Day", "S_L_45_Cysteine_c_Day", "S_L_45_Glutamine_c_Day", "S_L_45_Glutamate_c_Day", "S_Glycine_c_Day", "S_L_45_Proline_c_Day", "S_L_45_Tyrosine_c_Day", "S_L_45_Alanine_c_Day", "S_L_45_Asparagine_c_Day", "S_L_45_Serine_c_Day", "S_Orthophosphate_c_Day", "S_Starch_p_Day", "S_D_45_Fructose_c_Day", "S__40_S_41__45_Malate_c_Day", "S_Fumarate_c_Day", "S_Citrate_c_Day"]),
    PhotonReactionInhibitor(modelo, "Ex16_Night"),
    BiomassRegulator(modelo, "BIO_L_Day", "BIO_L_Night", ["R03845_c_Night", "R06267_c_Night", "R06286_c_Night", "R04801_c_Night", "R07856_c_Night", "R04803_c_Night", "R06961_c_Night", "R06960_c_Night", "V0004_Night", "R06284_c_Night", "R05618_c_Night", "R03824_c_Night", "R05341_c_Night", "R06965_c_Night", "R07558_c_Night", "R04798_c_Night", "R04787_c_Night", "R07840_c_Night", "R06963_c_Night", "R06962_c_Night", "R07850_c_Night", "R07530_c_Night"]),
    NitrateUptakeRatioCalibrator(modelo, "Ex4_Day", "Ex4_Night")
]

pipeline = Pipeline(modelo, Steps)
pipeline.run()

In [91]:
modelo.reactions.get_by_id("Biomass_Total").upper_bound = 0.11
modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0.11
modelo.objective = "Ex16_Day"
modelo.objective_direction = "max"

In [92]:
#modelo.reactions.get_by_id("Biomass_Total").upper_bound = 1000
#modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0
#modelo.objective = "Ex16_Day"
#modelo.objective_direction = "max"

In [93]:
#modelo.reactions.get_by_id("Biomass_Total").upper_bound = 0.11
#modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0.11
#modelo.objective = "Biomass_Total"
#modelo.objective_direction = "max"

In [94]:
#modelo.reactions.get_by_id("Biomass_Total").upper_bound = 1000
#modelo.reactions.get_by_id("Biomass_Total").lower_bound = 0
#modelo.objective = "Biomass_Total"
#modelo.objective_direction = "max"

In [95]:
from cobra.flux_analysis import pfba

solution = pfba(modelo)
R02732_c = solution.fluxes["R02732_c_Day"]
R01070N_c = solution.fluxes["R01070N_c_Day"]
R00762_c = solution.fluxes["R00762_c_Day"]
chex2 = round(((R02732_c - R01070N_c) + R00762_c) / 2, 2)
print(chex2)

9.31


In [96]:
R04779_p = solution.fluxes["R04779_p_Day"]
R00762_p = solution.fluxes["R00762_p_Day"]
R01070N_p = solution.fluxes["R01070N_p_Day"]

phex1 = round(((R04779_p - R00762_p) + R01070N_p) / 2, 2)
phex1

-39.42

In [97]:
R01061_c = solution.fluxes["R01061_c_Day"]
R01061_p = solution.fluxes["R01061_p_Day"]
R01063_p = solution.fluxes["R01063_p_Day"]
R01512_c = solution.fluxes["R01512_c_Day"]
R01518_c = solution.fluxes["R01518_c_Day"]
R01518_p = solution.fluxes["R01518_p_Day"]
R00658_c = solution.fluxes["R00658_c_Day"]
R00658_p = solution.fluxes["R00658_p_Day"]

Cphex = round(((R01061_c + R01061_p - R01063_p -R01512_c)/2 + (R01518_c + R01518_p) + (R00658_c + R00658_p))/3, 2)
Cphex #sem contar com R01512_d

39.48

In [98]:
R00200_c = solution.fluxes["R00200_c_Day"]

chex3 = round(R00200_c,2)
chex3

0.31

In [99]:
R00200_p = solution.fluxes["R00200_p_Day"]
R00206_p = solution.fluxes["R00206_p_Day"]

phex2 = R00200_p - R00206_p
phex2

0.0

In [100]:
R00351_m = solution.fluxes["R00351_m_Day"]
R00209_m = solution.fluxes["R00209_m_Day"]
TCM1 = solution.fluxes["TCM1_Day"]

tca2 = round((-TCM1 + R00351_m + R00209_m)/3,2)
tca2

0.0

In [101]:
import pandas as pd

dados = {'Valores Artigo (pFBA)': ['11.38', '1.719', '19.07', '0.0065', '5.40', '9.51'],
         'OF: max of Ex16_Day (pFBA)': ['9.31', '-39.42', '39.48', '0.31', '0.0', '0.0'],
         'OF: max of Ex16_Day (pFBA)*': ['9.86', '-40.54', '40.57', '0.16', '0.0', '-0.0'],
         'OF: max of Biomass_Total (pFBA)' : ['-0.0', '0.0', '0.32', '0.29', '0.0', '0.0'],
         'OF: max of Biomass_Total (pFBA)*' : ['-3.01', '0.0', '30.95', '16.33', '0.0', '0.0']}

df = pd.DataFrame(dados)

nomes = ['chex2', 'phex1', 'Cphex', 'chex3', 'phex2', 'tca2']
df.index = nomes

df

,Valores Artigo (pFBA),OF: max of Ex16_Day (pFBA),OF: max of Ex16_Day (pFBA)*,OF: max of Biomass_Total (pFBA),OF: max of Biomass_Total (pFBA)*
chex2,11.38,9.31,9.86,-0.0,-3.01
phex1,1.719,-39.42,-40.54,0.0,0.0
Cphex,19.07,39.48,40.57,0.32,30.95
chex3,0.0065,0.31,0.16,0.29,16.33
phex2,5.40,0.0,0.0,0.0,0.0
tca2,9.51,0.0,-0.0,0.0,0.0


*As constraints para a reação de Biomass_Total estão a (0, 1000) nestas hipóteses